In [1]:
import pandas as pd
import re
import multiprocessing

In [2]:
cpa_train_gt = pd.read_csv('SOTAB CPA/sotab_v2_cpa_training_set.csv')
cpa_test_gt = pd.read_csv('SOTAB CPA/sample_test.csv')

In [3]:
cpa_train_gt

,table_name,main_column_index,column_index,label
0,Book_11x17.pt_September2020_CPA.json.gz,0,3,datePublished
1,Book_11x17.pt_September2020_CPA.json.gz,0,1,isbn
2,Book_11x17.pt_September2020_CPA.json.gz,0,2,numberOfPages
3,Book_1jour-1jeu.com_September2020_CPA.json.gz,0,9,worstRating
4,Book_1jour-1jeu.com_September2020_CPA.json.gz,0,5,datePublished
...,...,...,...,...
109989,TVEpisode_yifytvseries.com_September2020_CPA.j...,0,3,datePublished
109990,TVEpisode_zazangels.com_September2020_CPA.json.gz,0,5,duration
109991,TVEpisode_zazangels.com_September2020_CPA.json.gz,0,1,description
109992,TVEpisode_zazangels.com_September2020_CPA.json.gz,0,2,url


In [4]:
cpa_test_gt

,table_name,main_column_index,column_index,label
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone
...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions


In [5]:
cpa_test_gt["class"] = cpa_test_gt["table_name"].apply(lambda x: x.split("_")[0])

In [6]:
cpa_test_gt

,table_name,main_column_index,column_index,label,class
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher,Book
1,Event_bfodurham.net_September2020_CPA.json.gz,0,3,eventStatus,Event
2,Event_bfodurham.net_September2020_CPA.json.gz,0,4,eventAttendanceMode,Event
3,Event_healthychelsea.org_September2020_CPA.jso...,0,4,organizer,Event
4,Event_healthychelsea.org_September2020_CPA.jso...,0,5,telephone,Event
...,...,...,...,...,...
504,Product_cit.li_September2020_CPA.json.gz,0,2,weight,Product
505,Product_cit.li_September2020_CPA.json.gz,0,4,productID,Product
506,Product_coininvest.com_September2020_CPA.json.gz,0,4,manufacturer,Product
507,Recipe_bakedbyanintrovert.com_September2020_CP...,0,11,recipeInstructions,Recipe


In [7]:
gt = {'train':{}, 'test':{}}
for index, row in cpa_train_gt.iterrows():
    if row["table_name"] not in gt['train']:
        gt['train'][row["table_name"]] = {}
    gt['train'][row["table_name"]][row["column_index"]] = row["label"]
    
for index, row in cpa_test_gt.iterrows():
    if row["table_name"] not in gt['test']:
        gt['test'][row["table_name"]] = {}
    gt['test'][row["table_name"]][row["column_index"]] = row["label"]

In [8]:
def clean_text(text):
        
    if(isinstance(text, dict)):
        text = ' '.join([ clean_text(v) for k, v in text.items()] )
    elif(isinstance(text, list)):
        text = map(clean_text, text)
        text = ' '.join(text)
        
    if pd.isnull(text):
        return ''
    
    text = re.sub(' +', ' ', str(text)).strip()
    
    return text

In [ ]:
""" #COLUMN PROPERTY MODIFIED

def get_all_table_columns(file_name, index):
    
    #By column
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
    else:
        path = 'SOTAB CPA/Test/'+file_name
    
    df = pd.read_json(path, compression='gzip', lines=True)
        
    cleaned_rows = []
    
    #Main column
    cleaned_main="{}{}"

    x = " ".join([" ".join(clean_text(row).split()[:20]) for row in df[0].tolist()[:5]]) #select 20 words
    cleaned_main=cleaned_main.format("Column 1: ", x)
    cleaned_rows.append(cleaned_main)

    cleaned_rows.append("\n Column 2:")
    
    
    for row in df.iloc[:, index].tolist():
        get_values_var=7
        cleaned = " ".join(clean_text(row).split()[:20]) #select 20 words
        
        if cleaned != "":
            cleaned_rows.append(cleaned)
        else:
            get_values_var=get_values_var-1

       
    
    return " ".join(cleaned_rows[:get_values_var]) #select rows """

In [11]:
def get_table(file_name):
    
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/'+file_name
        split = "train"
    else:
        path = 'SOTAB CPA/Test/'+file_name
        split = "test"

    df = pd.read_json(path, compression='gzip', lines=True) 
    labeled_idx = gt[split][file_name]
    df = df[[0]+list(labeled_idx.keys())]

    cleaned_rows = []


    for row in gt["test"]:
        col_labels = """"""
        cleaned_row = ''
        for i, l in enumerate(df[list(labeled_idx.keys())][:5]):
            col_labels += f"""Column {i+1}: {df[list(labeled_idx.keys())] }   \n"""
        cleaned_rows.append(col_labels.strip())

    return cleaned_rows

      

# Example usage
file_name = 'Book_1jour-1jeu.com_September2020_CPA.json.gz'
result = get_table(file_name)
print(result)

['Column 1:     9           5    6\n0   0  2019-06-21  EUR\n1   0  2020-08-28  EUR\n2   0  2019-09-20  EUR\n3   0  2019-03-08  EUR\n4   0  2019-10-02  EUR\n5   0  2019-02-15  EUR\n6   0  2019-10-18  EUR\n7   0  2019-10-18  EUR\n8   0  2019-11-22  EUR\n9   0  2019-05-10  EUR\n10  0  2019-10-18  EUR\n11  0  2019-03-08  EUR\n12  0  2019-09-20  EUR\n13  0  2019-02-15  EUR\n14  0  2019-12-20  EUR   \nColumn 2:     9           5    6\n0   0  2019-06-21  EUR\n1   0  2020-08-28  EUR\n2   0  2019-09-20  EUR\n3   0  2019-03-08  EUR\n4   0  2019-10-02  EUR\n5   0  2019-02-15  EUR\n6   0  2019-10-18  EUR\n7   0  2019-10-18  EUR\n8   0  2019-11-22  EUR\n9   0  2019-05-10  EUR\n10  0  2019-10-18  EUR\n11  0  2019-03-08  EUR\n12  0  2019-09-20  EUR\n13  0  2019-02-15  EUR\n14  0  2019-12-20  EUR   \nColumn 3:     9           5    6\n0   0  2019-06-21  EUR\n1   0  2020-08-28  EUR\n2   0  2019-09-20  EUR\n3   0  2019-03-08  EUR\n4   0  2019-10-02  EUR\n5   0  2019-02-15  EUR\n6   0  2019-10-18  EUR\n7 

In [15]:
def get_table(file_name):
    if file_name in cpa_train_gt["table_name"].tolist():
        path = 'SOTAB CPA/Train/' + file_name
        split = "train"
    else:
        path = 'SOTAB CPA/Test/' + file_name
        split = "test"

    df = pd.read_json(path, compression='gzip', lines=True) 
    labeled_idx = gt[split][file_name]
    df = df[[0] + list(labeled_idx.keys())]
   
    cleaned_rows = []

    col_labels = " || ".join(f"Column {i}" for i in range(1, len(df.columns)+1))
    cleaned_rows.append(f"{col_labels} || \n")

    cleaned_row = ''

    for _, row in df[:5].iterrows():
        if all(value is None for value in row):
            continue 
        cleaned_row += " || ".join(clean_text(row[col]) if row[col] is not None else '' for col in df.columns) + "||\n"
    if not cleaned_row.strip():
        for _, row in df[5:10].iterrows():
            if all(value is None for value in row):
                continue
            cleaned_row += " || ".join(clean_text(row[col]) if row[col] is not None else '' for col in df.columns) + "||\n"
    if cleaned_row.strip():
         cleaned_rows.append(cleaned_row)

    """ for _, row in df[:5].iterrows():
        cleaned_row = " || ".join(clean_text(str(row[col])) for col in df.columns) + "||\n"
        cleaned_rows.append(cleaned_row) """

    result = ''.join(cleaned_rows)

    return [result]



# Example usage
file_name = 'Book_1jour-1jeu.com_September2020_CPA.json.gz'
result = get_table(file_name)
print(result)


['Column 1 || Column 2 || Column 3 || Column 4 || \nEscape Game: Junior - La Malédiction de la Momie || 0 || 2019-06-21 || EUR||\nEscape Game: Serial Killer || 0 || 2020-08-28 || EUR||\nEscape Game: La Tour Noyée || 0 || 2019-09-20 || EUR||\nEscape Game: Junior - Opération Pizza || 0 || 2019-03-08 || EUR||\nEXIT: Le Livre - Journal de Bord 29 || 0 || 2019-10-02 || EUR||\n']


In [17]:
get_table('Book_antipodean.com_September2020_CPA.json.gz')

["Column 1 || Column 2 || Column 3 || Column 4 || \nOn Coxalgia, or Hip Disease. || Collins, || InStock || 65.0||\nSimonds Saws & Knives, Gold Medals and Highest Awards Everywhere. Atlanta 1895. Art Nouveau poster. || Livermore & Knight Co, ||  || ||\nNorth Island, New Zealand, Sheet 3, northeast corner from Tauranga to Dannevirke, folding map on linen. || Marcus F. Marks, Government Printer. || InStock || 85.0||\nTrade Receipts for wine and beer from Perth, Australia merchants 'H. Sherwood & Co., Wine & Spirit Merchants' and 'D. Nathan & Co., Bottle and Case Merchant'. ||  || InStock || 75.0||\nThe American Trans-Continental Route via New York Central & Hudson River R. R. and Connections, the Only 4 Track Line. New York to Omaha; the Union and Central Pacific Railways, Omaha to San Francisco; and the Pacific Mail Steamship Company's Lines, from San Francisco to destination. ||  || InStock || 375.0||\n"]

In [20]:
test_examples = []
for table in gt['test']:
    col_str = get_table(table)
    #test_examples.append([table, col_str, [""] + list(gt['test'][table].values()), table.split("_")[0]])
    test_examples.append([table, col_str, list(gt['test'][table].values()), table.split("_")[0]])

In [21]:
test_examples

[['Book_antipodean.com_September2020_CPA.json.gz',
  ["Column 1 || Column 2 || Column 3 || Column 4 || \nOn Coxalgia, or Hip Disease. || Collins, || InStock || 65.0||\nSimonds Saws & Knives, Gold Medals and Highest Awards Everywhere. Atlanta 1895. Art Nouveau poster. || Livermore & Knight Co, ||  || ||\nNorth Island, New Zealand, Sheet 3, northeast corner from Tauranga to Dannevirke, folding map on linen. || Marcus F. Marks, Government Printer. || InStock || 85.0||\nTrade Receipts for wine and beer from Perth, Australia merchants 'H. Sherwood & Co., Wine & Spirit Merchants' and 'D. Nathan & Co., Bottle and Case Merchant'. ||  || InStock || 75.0||\nThe American Trans-Continental Route via New York Central & Hudson River R. R. and Connections, the Only 4 Track Line. New York to Omaha; the Union and Central Pacific Railways, Omaha to San Francisco; and the Pacific Mail Steamship Company's Lines, from San Francisco to destination. ||  || InStock || 375.0||\n"],
  ['publisher', 'availabilit

In [22]:
len(test_examples)

125

In [23]:
cpa_test_gt[cpa_test_gt["table_name"] == "Book_antipodean.com_September2020_CPA.json.gz"]

,table_name,main_column_index,column_index,label,class
0,Book_antipodean.com_September2020_CPA.json.gz,0,4,publisher,Book
158,Book_antipodean.com_September2020_CPA.json.gz,0,3,availability,Book
159,Book_antipodean.com_September2020_CPA.json.gz,0,5,price,Book


In [29]:
train_examples = []
for table in gt['train']:
    col_str = get_table(table)
    #train_examples.append([table, col_str, [""] + list(gt['train'][table].values()), table.split("_")[0]])
    train_examples.append([table, col_str, list(gt['train'][table].values()), table.split("_")[0]])

In [31]:
len(train_examples)

29158

In [26]:
import pickle

In [32]:
file_name='SOTAB CPA/train-table-example.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()

In [33]:
file_name='SOTAB CPA/test-table-example.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()